In [71]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import display

In [72]:
#Substitua o link da sua busca aqui
completeUrl = 'https://bdtd.ibict.br/vufind/Search/Results?lookfor=atendimento+ubs&type=AllFields&filter%5B%5D=format%3A%22masterThesis%22'

In [73]:
maxQuantity = 25

splitDelimiter = '/vufind'
completeUrl = completeUrl.split(splitDelimiter, 1)

baseUrl = completeUrl[0]
query = splitDelimiter + completeUrl[1]

pageNumber = 1
researchCounter = 0

In [74]:
def getHTML(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    return soup

def getRowsInfos(soup, remainingQuantity):
    researchQuantityInPage = 0

    authors = soup.select('.author a')
    years = soup.find_all('div', {'id': 'datePublish'})
    titles = soup.find_all('a', {'class': 'title'}, limit=remainingQuantity)
    links = soup.find_all('a', {'class': 'fulltext'})

    rows = []

    for (author, year, title, link) in zip(authors, years, titles, links):
        author = author.text
        year = year.text.split()[-1]
        title = title.text.strip()
        link = link.get('href')

        rows.append({
          "Autor": author,
          "Ano de Defesa": year,
          "Título": title,
          "Link": link
        })

        researchQuantityInPage += 1

    return [researchQuantityInPage, rows]

def addNewRowToDf(rows, dfWithoutNewRow):
    dfWithNewRow = pd.DataFrame(rows).set_index("Autor")

    dfWithNewRow = pd.concat([dfWithoutNewRow, dfWithNewRow])

    return dfWithNewRow

In [75]:
url = baseUrl + query
soup = getHTML(url)

df = pd.DataFrame()

In [76]:
while researchCounter < maxQuantity:
  remaining = maxQuantity - researchCounter
  quantityAdded = 0
  if pageNumber == 1:
    print("Página:", pageNumber)

    [quantityAdded, rows] = getRowsInfos(soup, remaining)

    df = addNewRowToDf(rows, df)

    print("Trabalhos coletados:", quantityAdded)
  elif remaining > 0:
    print("Página:", pageNumber)

    try:
      nextQueryLink = soup.select('ul.pagination > li.active + li > a')[0].get('href')
      url = baseUrl + nextQueryLink
      soup = getHTML(url)
    except:
      print(f"Parece que a sua busca não retorna {maxQuantity} resultados")
      break

    [quantityAdded, rows] = getRowsInfos(soup, remaining)

    df = addNewRowToDf(rows, df)

    print("Trabalhos coletados:", quantityAdded)

  pageNumber = pageNumber + 1
  researchCounter += quantityAdded


Página: 1
Trabalhos coletados: 20
Página: 2
Trabalhos coletados: 5


In [77]:
print(f"Foram coletados {researchCounter} trabalhos.")
df.to_excel("output.xlsx")
print("Seu arquivo está pronto!")
print("É possível acessá-lo no menu a esquerda.")

Foram coletados 25 trabalhos.
Seu arquivo está pronto!
É possível acessá-lo no menu a esquerda.


In [78]:
df

,Ano de Defesa,Título,Link
Autor,,,
"Sarmento, Maria das Graças Silva",2019,Avaliação da saúde bucal de pacientes portador...,https://www.arca.fiocruz.br/handle/icict/32099
"Gurgel, Camila Augusta de Rubim Costa",2014,Investigando o cuidado em equipe de enfermagem...,https://repositorio.ufrn.br/jspui/handle/12345...
"Horta, Heloisa Helena Lemos",2007,Subsídios para atenção integral do idoso: perf...,https://repositorio.cruzeirodosul.edu.br/handl...
"Horta, Heloisa Helena Lemos",2007,Subsídios para atenção integral do idoso: perf...,https://repositorio.cruzeirodosul.edu.br/handl...
Marco Antonio Kulik,2015,Como os integrantes profissionais de uma Unida...,https://doi.org/10.11606/D.23.2015.tde-1706201...
Silvana Aparecida Sanavio,2016,Elaboração de material educativo para ações de...,https://doi.org/10.11606/D.108.2016.tde-211120...
Carolina Pinheiro Perracini,2009,Promoção da saúde e o desenvolvimento de habil...,https://doi.org/10.11606/D.5.2009.tde-28082009...
"Fernandes, Alexandre",2013,Análise das informações a respeito das leishma...,https://repositorio.ufsc.br/xmlui/handle/12345...
"Medeiros, Deisyanne Camara Alves de",2012,UBS RN-7: proposta arquitetônica para um padrã...,https://repositorio.ufrn.br/jspui/handle/12345...
